In [1]:
import lmdb
import os
import torch
import random
import skimage
import numpy as np
import torch.nn as nn
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

print("complete")

complete


In [3]:
train_vectors = np.load("part_bird_train.npy")
test_vectors = np.load("part_bird_test.npy")

In [6]:
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]
        new_vectors = []
        for i in range(self.part_num):
            vectors_part = vectors[i]
            if isinstance(vectors_part, list):
                vectors_part = np.array(vectors_part)
            num_p = vectors_part.shape[0]
#             print(num_p)
            cls_p = 0
            lists = []
            lab_set = {i for i in range(num_p)}
            while(cls_p < num_p):
                cls_l = []
                start = lab_set.pop()
                cls_l.append(start)
                cls_p += 1
                cls_new_p = True
                while(cls_new_p):
                    rm_l = []
                    cls_new_p = False
                    for k in lab_set:
                        new_p = False
                        for t in range(len(cls_l)):
#                             print(pow(vectors_part[k, 0:2] - vectors_part[cls_l[t], 0:2], 2))
                            dis = np.sum(pow(vectors_part[k, 0:2] - vectors_part[cls_l[t], 0:2], 2)) 
                            if dis <= 2:
                                new_p = True
                                break
                        if new_p:
                            cls_new_p = True
                            rm_l.append(k)
                            cls_l.append(k)
                            cls_p += 1

                    for kk in range(len(rm_l)):
                        lab_set.remove(rm_l[kk])
                lists.append(cls_l)
#             print(len(lists))
            l_n = [len(lists[i]) for i in range(len(lists))]
#             print(l_n)
            new_vector_index = lists[int(l_n.index(max(l_n)))]
            new_vector = np.zeros((max(l_n), 4))
            for i in range(max(l_n)):
                new_vector[i] = vectors_part[new_vector_index[i]]
            new_vectors.append(new_vector)
        new_vectors = np.array(new_vectors)
        
        weights = np.zeros(self.part_num)
        points = np.zeros((self.part_num, 2))
        for i in range(self.part_num):
            vectors_part = new_vectors[i]
            if isinstance(vectors_part, list):
                vectors_part = np.array(vectors_part)
            weights[i] = np.sum(vectors_part[:, 2])/vectors_part.shape[0]
            points[i, 0] = np.sum(vectors_part[:, 0])/vectors_part.shape[0]
            points[i, 1] = np.sum(vectors_part[:, 1])/vectors_part.shape[0]
            
        index = np.argsort(weights)

        for i in range(self.part_num):
            vectors_part = new_vectors[index[i]]
            if isinstance(vectors_part, list):
                vectors_part = np.array(vectors_part)

            x, y = np.round(points[index[i], 0] * 16 + 8), np.round(points[index[i], 1] * 16 + 8)
            
            l_d = np.maximum((np.max(vectors_part[:, 0]) - np.min(vectors_part[:, 0])), \
                            (np.max(vectors_part[:, 1]) - np.min(vectors_part[:, 1])))

            if l_d <= 4:
                l = 48
            elif 4 < l_d <= 6:
                l = 64
            elif 6 < l_d <= 8:
                l = 80
            elif 8 < l_d <= 10:
                l = 96
            elif 10 < l_d <= 12:
                l = 112
            elif 12 < l_d <= 14:
                l = 128
            else:
                l = 144

            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))

            sample['image%d' % (self.part_num - i - 1)] = image[m1:m2, n1:n2]            
        
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])

        return sample

In [7]:
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
            ])

root_dir = '/home/lvlab/Pytorch/images'
train_file = '/home/lvlab/Pytorch/train.txt'
test_file = '/home/lvlab/Pytorch/test.txt'

img_data = {'train': Bird_Dataset_Process(train_file, root_dir, train_vectors, 4, data_transform),
                  'test': Bird_Dataset_Process(test_file, root_dir, test_vectors, 4, data_transform)}
loader_img = {x: torch.utils.data.DataLoader(img_data[x], batch_size=20,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(img_data[x]) for x in ['train', 'test']}
print(img_data['test'][0])

{'image2': tensor([[[-0.8893, -0.8722, -0.7865,  ..., -0.3242, -0.3070, -0.3070],
         [-0.8550, -0.8379, -0.7694,  ..., -0.3242, -0.3070, -0.3070],
         [-0.7694, -0.7694, -0.7352,  ..., -0.3070, -0.2899, -0.2899],
         ...,
         [-0.1872, -0.1872, -0.1872,  ..., -2.0709, -2.0709, -2.0709],
         [-0.2043, -0.2043, -0.2043,  ..., -2.0538, -2.0709, -2.0709],
         [-0.2043, -0.2043, -0.2214,  ..., -2.0538, -2.0709, -2.0709]],

        [[-1.3637, -1.3463, -1.2940,  ...,  0.2049,  0.2223,  0.2223],
         [-1.3289, -1.3115, -1.2766,  ...,  0.2049,  0.2223,  0.2223],
         [-1.2417, -1.2417, -1.2417,  ...,  0.2223,  0.2397,  0.2397],
         ...,
         [ 0.3095,  0.3095,  0.3269,  ..., -0.8757, -0.8932, -0.8932],
         [ 0.3269,  0.3269,  0.3269,  ..., -0.8583, -0.8757, -0.8757],
         [ 0.3269,  0.3269,  0.3269,  ..., -0.8583, -0.8757, -0.8757]],

        [[-0.8374, -0.8225, -0.7927,  ...,  0.9817,  0.9817,  0.9817],
         [-0.8076, -0.7927, -0.777

/home/lvlab/.tensorflow_pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/lvlab/.tensorflow_pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/lvlab/.tensorflow_pytorch/lib/python3.5/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
                
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(loader_img[phase]):
                img0 = sample_batched['image0']
                img1 = sample_batched['image1']
                img2 = sample_batched['image2']
                img3 = sample_batched['image3']
                labels = sample_batched['label']
                
                img0 = img0.to(device)
                img1 = img1.to(device)
                img2 = img2.to(device)
                img3 = img3.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    output0 = model(img0)
                    output1 = model(img1)
                    output2 = model(img2)
                    output3 = model(img3)
                    outputs = output0 + output1 + output2 + output3
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * img1.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [9]:
import time
import copy
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:1")

vgg_16 = models.vgg16(pretrained=True)
vgg_16.classifier = nn.Sequential(*list(vgg_16.classifier.children())[:1])
vgg_16.features[30] = nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
vgg_16.features.add_module('31', nn.Dropout(0.8)) 
vgg_16.classifier[0] = nn.Linear(in_features=512, out_features=200, bias=True)
# vgg_16 = nn.DataParallel(vgg_16, device_ids=[2, 3])
vgg_16 = vgg_16.to(device)


criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg_16.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=5)

model = train_model(vgg_16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=60)

Epoch 0/59
----------
***********************
learning rate = 0.001000
Iteration 0, loss = 5.545182
Iteration 50, loss = 5.281681
Iteration 100, loss = 5.306508
Iteration 150, loss = 5.294640
Iteration 200, loss = 5.264226
Iteration 250, loss = 5.142363
train Loss: 5.2787 Acc: 0.0093
train 1 epoch time: 3.88min
test Loss: 5.0802 Acc: 0.0271
test 1 epoch time: 1.99min

Epoch 1/59
----------
Iteration 0, loss = 5.185041
Iteration 50, loss = 4.990376
Iteration 100, loss = 4.987313
Iteration 150, loss = 4.592115
Iteration 200, loss = 4.405859
Iteration 250, loss = 3.599209
train Loss: 4.6330 Acc: 0.0671
train 1 epoch time: 4.02min
test Loss: 3.6179 Acc: 0.2133
test 1 epoch time: 1.96min

Epoch 2/59
----------
Iteration 0, loss = 4.010584
Iteration 50, loss = 4.016866
Iteration 100, loss = 3.425609
Iteration 150, loss = 3.883745
Iteration 200, loss = 3.106911
Iteration 250, loss = 3.049938
train Loss: 3.4434 Acc: 0.2129
train 1 epoch time: 4.01min
test Loss: 2.3990 Acc: 0.4270
test 1 epoch 

train Loss: 0.1339 Acc: 0.9576
train 1 epoch time: 3.98min
test Loss: 0.9753 Acc: 0.8098
test 1 epoch time: 1.92min

Epoch 25/59
----------
***********************
learning rate = 0.000500
Iteration 0, loss = 0.006133
Iteration 50, loss = 0.178026
Iteration 100, loss = 0.114911
Iteration 150, loss = 0.030388
Iteration 200, loss = 0.121490
Iteration 250, loss = 0.087371
train Loss: 0.1267 Acc: 0.9596
train 1 epoch time: 3.99min
test Loss: 1.0358 Acc: 0.8015
test 1 epoch time: 1.94min

Epoch 26/59
----------
Iteration 0, loss = 0.156917
Iteration 50, loss = 0.100641
Iteration 100, loss = 0.083963
Iteration 150, loss = 0.007397
Iteration 200, loss = 0.028196
Iteration 250, loss = 0.069099
train Loss: 0.1256 Acc: 0.9610
train 1 epoch time: 4.00min
test Loss: 1.0027 Acc: 0.8007
test 1 epoch time: 1.95min

Epoch 27/59
----------
Iteration 0, loss = 0.180071
Iteration 50, loss = 0.042499
Iteration 100, loss = 0.406288
Iteration 150, loss = 0.065361
Iteration 200, loss = 0.109447
Iteration 250

Iteration 100, loss = 0.000732
Iteration 150, loss = 0.295150
Iteration 200, loss = 0.041207
Iteration 250, loss = 0.042267
train Loss: 0.0305 Acc: 0.9918
train 1 epoch time: 3.96min
test Loss: 1.1326 Acc: 0.8241
test 1 epoch time: 1.39min

Epoch 50/59
----------
***********************
learning rate = 0.000031
Iteration 0, loss = 0.001726
Iteration 50, loss = 0.013185
Iteration 100, loss = 0.001734
Iteration 150, loss = 0.004486
Iteration 200, loss = 0.023944
Iteration 250, loss = 0.004339
train Loss: 0.0274 Acc: 0.9918
train 1 epoch time: 3.95min
test Loss: 1.1379 Acc: 0.8250
test 1 epoch time: 1.43min

Epoch 51/59
----------
Iteration 0, loss = 0.015521
Iteration 50, loss = 0.000888
Iteration 100, loss = 0.014806
Iteration 150, loss = 0.004018
Iteration 200, loss = 0.035382
Iteration 250, loss = 0.000687
train Loss: 0.0312 Acc: 0.9898
train 1 epoch time: 3.96min
test Loss: 1.1365 Acc: 0.8252
test 1 epoch time: 1.34min

Epoch 52/59
----------
Iteration 0, loss = 0.004513
Iteration 50

In [10]:
torch.save(model, "/home/lvlab/Pytorch/part_bird")